In [1]:
# https://deeplearningcourses.com/c/deep-reinforcement-learning-in-python
# https://www.udemy.com/deep-reinforcement-learning-in-python
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future

import copy
import gym
import os
import sys
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from gym import wrappers
from datetime import datetime
from scipy.misc import imresize





##### testing only
# MAX_EXPERIENCES = 10000
# MIN_EXPERIENCES = 1000


MAX_EXPERIENCES = 500000
MIN_EXPERIENCES = 50000
TARGET_UPDATE_PERIOD = 10000
IM_SIZE = 80
K = 4 #env.action_space.n




def downsample_image(A):
  B = A[31:195] # select the important parts of the image
  B = B.mean(axis=2) # convert to grayscale

  # downsample image
  # changing aspect ratio doesn't significantly distort the image
  # nearest neighbor interpolation produces a much sharper image
  # than default bilinear
  B = imresize(B, size=(IM_SIZE, IM_SIZE), interp='nearest')
  return B


def update_state(state, obs):
  obs_small = downsample_image(obs)
  return np.append(state[1:], np.expand_dims(obs_small, 0), axis=0)


class DQN:
  def __init__(self, K, conv_layer_sizes, hidden_layer_sizes, gamma, scope):

    self.K = K
    self.scope = scope

    with tf.variable_scope(scope):

      # inputs and targets
      self.X = tf.placeholder(tf.float32, shape=(None, 4, IM_SIZE, IM_SIZE), name='X')

      # tensorflow convolution needs the order to be:
      # (num_samples, height, width, "color")
      # so we need to tranpose later
      self.G = tf.placeholder(tf.float32, shape=(None,), name='G')
      self.actions = tf.placeholder(tf.int32, shape=(None,), name='actions')

      # calculate output and cost
      # convolutional layers
      # these built-in layers are faster and don't require us to
      # calculate the size of the output of the final conv layer!
      Z = self.X / 255.0
      Z = tf.transpose(Z, [0, 2, 3, 1])
      for num_output_filters, filtersz, poolsz in conv_layer_sizes:
        Z = tf.contrib.layers.conv2d(
          Z,
          num_output_filters,
          filtersz,
          poolsz,
          activation_fn=tf.nn.relu
        )

      # fully connected layers
      Z = tf.contrib.layers.flatten(Z)
      for M in hidden_layer_sizes:
        Z = tf.contrib.layers.fully_connected(Z, M)

      # final output layer
      self.predict_op = tf.contrib.layers.fully_connected(Z, K)

      selected_action_values = tf.reduce_sum(
        self.predict_op * tf.one_hot(self.actions, K),
        reduction_indices=[1]
      )

      cost = tf.reduce_mean(tf.square(self.G - selected_action_values))
      # self.train_op = tf.train.AdamOptimizer(1e-2).minimize(cost)
      # self.train_op = tf.train.AdagradOptimizer(1e-2).minimize(cost)
      # self.train_op = tf.train.RMSPropOptimizer(2.5e-4, decay=0.99, epsilon=10e-3).minimize(cost)
      self.train_op = tf.train.RMSPropOptimizer(0.00025, 0.99, 0.0, 1e-6).minimize(cost)
      # self.train_op = tf.train.MomentumOptimizer(1e-3, momentum=0.9).minimize(cost)
      # self.train_op = tf.train.GradientDescentOptimizer(1e-4).minimize(cost)

      self.cost = cost

  def copy_from(self, other):
    mine = [t for t in tf.trainable_variables() if t.name.startswith(self.scope)]
    mine = sorted(mine, key=lambda v: v.name)
    theirs = [t for t in tf.trainable_variables() if t.name.startswith(other.scope)]
    theirs = sorted(theirs, key=lambda v: v.name)

    ops = []
    for p, q in zip(mine, theirs):
      actual = self.session.run(q)
      op = p.assign(actual)
      ops.append(op)

    self.session.run(ops)

  def set_session(self, session):
    self.session = session

  def predict(self, states):
    return self.session.run(self.predict_op, feed_dict={self.X: states})

  def update(self, states, actions, targets):
    c, _ = self.session.run(
      [self.cost, self.train_op],
      feed_dict={
        self.X: states,
        self.G: targets,
        self.actions: actions
      }
    )
    return c

  def sample_action(self, x, eps):
    if np.random.random() < eps:
      return np.random.choice(self.K)
    else:
      return np.argmax(self.predict([x])[0])


def learn(model, target_model, experience_replay_buffer, gamma, batch_size):
  # Sample experiences
  samples = random.sample(experience_replay_buffer, batch_size)
  states, actions, rewards, next_states, dones = map(np.array, zip(*samples))

  # Calculate targets
  next_Qs = target_model.predict(next_states)
  next_Q = np.amax(next_Qs, axis=1)
  targets = rewards + np.invert(dones).astype(np.float32) * gamma * next_Q

  # Update model
  loss = model.update(states, actions, targets)
  return loss


def play_one(
  env,
  total_t,
  experience_replay_buffer,
  model,
  target_model,
  gamma,
  batch_size,
  epsilon,
  epsilon_change,
  epsilon_min):

  t0 = datetime.now()

  # Reset the environment
  obs = env.reset()
  obs_small = downsample_image(obs)
  state = np.stack([obs_small] * 4, axis=0)
  assert(state.shape == (4, 80, 80))
  loss = None


  total_time_training = 0
  num_steps_in_episode = 0
  episode_reward = 0

  done = False
  while not done:

    # Update target network
    if total_t % TARGET_UPDATE_PERIOD == 0:
      target_model.copy_from(model)
      print("Copied model parameters to target network. total_t = %s, period = %s" % (total_t, TARGET_UPDATE_PERIOD))


    # Take action
    action = model.sample_action(state, epsilon)
    obs, reward, done, _ = env.step(action)
    obs_small = downsample_image(obs)
    next_state = np.append(state[1:], np.expand_dims(obs_small, 0), axis=0)
    # assert(state.shape == (4, 80, 80))



    episode_reward += reward

    # Remove oldest experience if replay buffer is full
    if len(experience_replay_buffer) == MAX_EXPERIENCES:
      experience_replay_buffer.pop(0)

    # Save the latest experience
    experience_replay_buffer.append((state, action, reward, next_state, done))

    # Train the model, keep track of time
    t0_2 = datetime.now()
    loss = learn(model, target_model, experience_replay_buffer, gamma, batch_size)
    dt = datetime.now() - t0_2

    total_time_training += dt.total_seconds()
    num_steps_in_episode += 1


    state = next_state
    total_t += 1

    epsilon = max(epsilon - epsilon_change, epsilon_min)

  return total_t, episode_reward, (datetime.now() - t0), num_steps_in_episode, total_time_training/num_steps_in_episode, epsilon



def main():

  # hyperparams and initialize stuff
  conv_layer_sizes = [(32, 8, 4), (64, 4, 2), (64, 3, 1)]
  hidden_layer_sizes = [512]
  gamma = 0.99
  batch_sz = 32
  num_episodes = 10000
  total_t = 0
  experience_replay_buffer = []
  episode_rewards = np.zeros(num_episodes)



  # epsilon
  # decays linearly until 0.1
  epsilon = 1.0
  epsilon_min = 0.1
  epsilon_change = (epsilon - epsilon_min) / 500000



  # Create environment
  env = gym.envs.make("Breakout-v0")
 


  # Create models
  model = DQN(
    K=K,
    conv_layer_sizes=conv_layer_sizes,
    hidden_layer_sizes=hidden_layer_sizes,
    gamma=gamma,
    scope="model")
  target_model = DQN(
    K=K,
    conv_layer_sizes=conv_layer_sizes,
    hidden_layer_sizes=hidden_layer_sizes,
    gamma=gamma,
    scope="target_model"
  )



  with tf.Session() as sess:
    model.set_session(sess)
    target_model.set_session(sess)
    sess.run(tf.global_variables_initializer())


    print("Populating experience replay buffer...")
    obs = env.reset()
    obs_small = downsample_image(obs)
    state = np.stack([obs_small] * 4, axis=0)
    # assert(state.shape == (4, 80, 80))
    for i in range(MIN_EXPERIENCES):

        action = np.random.choice(K)
        obs, reward, done, _ = env.step(action)
        next_state = update_state(state, obs)
        # assert(state.shape == (4, 80, 80))
        experience_replay_buffer.append((state, action, reward, next_state, done))

        if done:
            obs = env.reset()
            obs_small = downsample_image(obs)
            state = np.stack([obs_small] * 4, axis=0)
            # assert(state.shape == (4, 80, 80))
        else:
            state = next_state


    # Play a number of episodes and learn!
    for i in range(num_episodes):

      total_t, episode_reward, duration, num_steps_in_episode, time_per_step, epsilon = play_one(
        env,
        total_t,
        experience_replay_buffer,
        model,
        target_model,
        gamma,
        batch_sz,
        epsilon,
        epsilon_change,
        epsilon_min,
      )
      episode_rewards[i] = episode_reward

      last_100_avg = episode_rewards[max(0, i - 100):i + 1].mean()
      print("Episode:", i,
        "Duration:", duration,
        "Num steps:", num_steps_in_episode,
        "Reward:", episode_reward,
        "Training time per step:", "%.3f" % time_per_step,
        "Avg Reward (Last 100):", "%.3f" % last_100_avg,
        "Epsilon:", "%.3f" % epsilon
      )
      sys.stdout.flush()

In [ ]:
main()

Populating experience replay buffer...
Copied model parameters to target network. total_t = 0, period = 10000
Episode: 0 Duration: 0:00:03.988829 Num steps: 199 Reward: 1.0 Training time per step: 0.017 Avg Reward (Last 100): 1.000 Epsilon: 1.000
Episode: 1 Duration: 0:00:02.731743 Num steps: 241 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.000 Epsilon: 0.999
Episode: 2 Duration: 0:00:02.923278 Num steps: 256 Reward: 1.0 Training time per step: 0.010 Avg Reward (Last 100): 1.000 Epsilon: 0.999
Episode: 3 Duration: 0:00:02.612389 Num steps: 229 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.000 Epsilon: 0.998
Episode: 4 Duration: 0:00:01.889264 Num steps: 165 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 0.800 Epsilon: 0.998
Episode: 5 Duration: 0:00:05.518013 Num steps: 489 Reward: 9.0 Training time per step: 0.009 Avg Reward (Last 100): 2.167 Epsilon: 0.997
Episode: 6 Duration: 0:00:02.779699 Num steps: 246 Reward: 2.0 Train

Episode: 59 Duration: 0:00:01.870559 Num steps: 164 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.333 Epsilon: 0.973
Episode: 60 Duration: 0:00:02.781739 Num steps: 244 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.328 Epsilon: 0.973
Episode: 61 Duration: 0:00:02.640798 Num steps: 233 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.323 Epsilon: 0.973
Episode: 62 Duration: 0:00:03.163880 Num steps: 280 Reward: 2.0 Training time per step: 0.009 Avg Reward (Last 100): 1.333 Epsilon: 0.972
Episode: 63 Duration: 0:00:01.945819 Num steps: 171 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.312 Epsilon: 0.972
Episode: 64 Duration: 0:00:02.952333 Num steps: 259 Reward: 2.0 Training time per step: 0.009 Avg Reward (Last 100): 1.323 Epsilon: 0.971
Episode: 65 Duration: 0:00:02.008837 Num steps: 176 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.303 Epsilon: 0.971
Episode: 66 Duration: 0:00:01.9331

Episode: 118 Duration: 0:00:02.641456 Num steps: 231 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.287 Epsilon: 0.948
Episode: 119 Duration: 0:00:03.021705 Num steps: 265 Reward: 2.0 Training time per step: 0.009 Avg Reward (Last 100): 1.297 Epsilon: 0.947
Episode: 120 Duration: 0:00:02.654519 Num steps: 233 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.297 Epsilon: 0.947
Episode: 121 Duration: 0:00:03.214489 Num steps: 281 Reward: 2.0 Training time per step: 0.009 Avg Reward (Last 100): 1.317 Epsilon: 0.946
Episode: 122 Duration: 0:00:02.159770 Num steps: 188 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.317 Epsilon: 0.946
Copied model parameters to target network. total_t = 30000, period = 10000
Episode: 123 Duration: 0:00:03.319191 Num steps: 275 Reward: 2.0 Training time per step: 0.010 Avg Reward (Last 100): 1.317 Epsilon: 0.946
Episode: 124 Duration: 0:00:05.855161 Num steps: 513 Reward: 6.0 Training time per step: 0.

Episode: 176 Duration: 0:00:04.226740 Num steps: 367 Reward: 3.0 Training time per step: 0.010 Avg Reward (Last 100): 1.198 Epsilon: 0.922
Episode: 177 Duration: 0:00:02.667795 Num steps: 232 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.198 Epsilon: 0.922
Episode: 178 Duration: 0:00:02.464492 Num steps: 216 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.208 Epsilon: 0.922
Episode: 179 Duration: 0:00:03.253854 Num steps: 283 Reward: 2.0 Training time per step: 0.010 Avg Reward (Last 100): 1.228 Epsilon: 0.921
Episode: 180 Duration: 0:00:03.878109 Num steps: 340 Reward: 3.0 Training time per step: 0.009 Avg Reward (Last 100): 1.248 Epsilon: 0.920
Episode: 181 Duration: 0:00:02.464377 Num steps: 214 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.228 Epsilon: 0.920
Episode: 182 Duration: 0:00:02.055880 Num steps: 179 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.208 Epsilon: 0.920
Episode: 183 Duration: 0:00

Episode: 235 Duration: 0:00:02.006951 Num steps: 174 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.277 Epsilon: 0.896
Episode: 236 Duration: 0:00:01.818981 Num steps: 159 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.277 Epsilon: 0.896
Episode: 237 Duration: 0:00:02.424067 Num steps: 211 Reward: 1.0 Training time per step: 0.009 Avg Reward (Last 100): 1.257 Epsilon: 0.895
Episode: 238 Duration: 0:00:03.367845 Num steps: 295 Reward: 2.0 Training time per step: 0.009 Avg Reward (Last 100): 1.267 Epsilon: 0.895
Episode: 239 Duration: 0:00:03.834287 Num steps: 333 Reward: 3.0 Training time per step: 0.009 Avg Reward (Last 100): 1.277 Epsilon: 0.894
Episode: 240 Duration: 0:00:02.880398 Num steps: 249 Reward: 2.0 Training time per step: 0.009 Avg Reward (Last 100): 1.267 Epsilon: 0.894
Episode: 241 Duration: 0:00:02.068585 Num steps: 179 Reward: 0.0 Training time per step: 0.009 Avg Reward (Last 100): 1.257 Epsilon: 0.894
Episode: 242 Duration: 0:00

...

Episode: 1851 Duration: 0:00:17.220846 Num steps: 1351 Reward: 35.0 Training time per step: 0.010 Avg Reward (Last 100): 23.980 Epsilon: 0.100
Episode: 1852 Duration: 0:00:17.825846 Num steps: 1394 Reward: 42.0 Training time per step: 0.010 Avg Reward (Last 100): 24.109 Epsilon: 0.100
Episode: 1853 Duration: 0:00:15.972649 Num steps: 1249 Reward: 34.0 Training time per step: 0.010 Avg Reward (Last 100): 24.089 Epsilon: 0.100
Episode: 1854 Duration: 0:00:09.175234 Num steps: 719 Reward: 15.0 Training time per step: 0.010 Avg Reward (Last 100): 23.950 Epsilon: 0.100
Episode: 1855 Duration: 0:00:13.729367 Num steps: 1078 Reward: 26.0 Training time per step: 0.010 Avg Reward (Last 100): 23.970 Epsilon: 0.100
Episode: 1856 Duration: 0:00:12.731541 Num steps: 1000 Reward: 21.0 Training time per step: 0.010 Avg Reward (Last 100): 24.000 Epsilon: 0.100
Episode: 1857 Duration: 0:00:11.156630 Num steps: 878 Reward: 16.0 Training time per step: 0.009 Avg Reward (Last 100): 23.861 Epsilon: 0.100
E

Episode: 1906 Duration: 0:00:12.588806 Num steps: 984 Reward: 34.0 Training time per step: 0.010 Avg Reward (Last 100): 22.505 Epsilon: 0.100
Episode: 1907 Duration: 0:00:08.955004 Num steps: 703 Reward: 11.0 Training time per step: 0.010 Avg Reward (Last 100): 22.297 Epsilon: 0.100
Copied model parameters to target network. total_t = 760000, period = 10000
Episode: 1908 Duration: 0:00:12.219150 Num steps: 925 Reward: 19.0 Training time per step: 0.010 Avg Reward (Last 100): 22.248 Epsilon: 0.100
Episode: 1909 Duration: 0:00:15.979255 Num steps: 1246 Reward: 32.0 Training time per step: 0.010 Avg Reward (Last 100): 22.228 Epsilon: 0.100
Episode: 1910 Duration: 0:00:18.331834 Num steps: 1432 Reward: 37.0 Training time per step: 0.010 Avg Reward (Last 100): 22.327 Epsilon: 0.100
Episode: 1911 Duration: 0:00:10.551604 Num steps: 824 Reward: 13.0 Training time per step: 0.010 Avg Reward (Last 100): 22.218 Epsilon: 0.100
Episode: 1912 Duration: 0:00:10.209470 Num steps: 802 Reward: 16.0 Tra

Episode: 1961 Duration: 0:00:13.688556 Num steps: 1061 Reward: 27.0 Training time per step: 0.010 Avg Reward (Last 100): 23.277 Epsilon: 0.100
Episode: 1962 Duration: 0:00:09.722973 Num steps: 757 Reward: 11.0 Training time per step: 0.010 Avg Reward (Last 100): 23.119 Epsilon: 0.100
Copied model parameters to target network. total_t = 820000, period = 10000
Episode: 1963 Duration: 0:00:14.344877 Num steps: 1083 Reward: 21.0 Training time per step: 0.010 Avg Reward (Last 100): 23.158 Epsilon: 0.100
Episode: 1964 Duration: 0:00:18.074968 Num steps: 1404 Reward: 39.0 Training time per step: 0.010 Avg Reward (Last 100): 23.426 Epsilon: 0.100
Episode: 1965 Duration: 0:00:14.003575 Num steps: 1087 Reward: 26.0 Training time per step: 0.010 Avg Reward (Last 100): 23.436 Epsilon: 0.100
Episode: 1966 Duration: 0:00:15.149287 Num steps: 1177 Reward: 28.0 Training time per step: 0.010 Avg Reward (Last 100): 23.525 Epsilon: 0.100
Episode: 1967 Duration: 0:00:09.085554 Num steps: 705 Reward: 12.0 

Episode: 2015 Duration: 0:00:18.549825 Num steps: 1406 Reward: 34.0 Training time per step: 0.010 Avg Reward (Last 100): 26.337 Epsilon: 0.100
Episode: 2016 Duration: 0:00:19.625550 Num steps: 1526 Reward: 35.0 Training time per step: 0.010 Avg Reward (Last 100): 26.564 Epsilon: 0.100
Episode: 2017 Duration: 0:00:13.378748 Num steps: 1042 Reward: 17.0 Training time per step: 0.010 Avg Reward (Last 100): 26.347 Epsilon: 0.100
Episode: 2018 Duration: 0:00:16.488992 Num steps: 1280 Reward: 40.0 Training time per step: 0.010 Avg Reward (Last 100): 26.535 Epsilon: 0.100
Episode: 2019 Duration: 0:00:14.927117 Num steps: 1160 Reward: 29.0 Training time per step: 0.010 Avg Reward (Last 100): 26.693 Epsilon: 0.100
Episode: 2020 Duration: 0:00:09.041398 Num steps: 704 Reward: 14.0 Training time per step: 0.010 Avg Reward (Last 100): 26.475 Epsilon: 0.100
Episode: 2021 Duration: 0:00:14.166178 Num steps: 1101 Reward: 23.0 Training time per step: 0.010 Avg Reward (Last 100): 26.436 Epsilon: 0.100


Episode: 2070 Duration: 0:00:13.684662 Num steps: 1064 Reward: 21.0 Training time per step: 0.010 Avg Reward (Last 100): 26.634 Epsilon: 0.100
Episode: 2071 Duration: 0:00:16.947446 Num steps: 1313 Reward: 39.0 Training time per step: 0.010 Avg Reward (Last 100): 26.723 Epsilon: 0.100
Episode: 2072 Duration: 0:00:12.281502 Num steps: 954 Reward: 21.0 Training time per step: 0.010 Avg Reward (Last 100): 26.446 Epsilon: 0.100
Episode: 2073 Duration: 0:00:10.250245 Num steps: 798 Reward: 16.0 Training time per step: 0.010 Avg Reward (Last 100): 26.208 Epsilon: 0.100
Episode: 2074 Duration: 0:00:08.934188 Num steps: 696 Reward: 14.0 Training time per step: 0.010 Avg Reward (Last 100): 26.178 Epsilon: 0.100
Episode: 2075 Duration: 0:00:11.916310 Num steps: 925 Reward: 18.0 Training time per step: 0.010 Avg Reward (Last 100): 26.158 Epsilon: 0.100
Episode: 2076 Duration: 0:00:11.963178 Num steps: 928 Reward: 16.0 Training time per step: 0.010 Avg Reward (Last 100): 26.188 Epsilon: 0.100
Epis